# Iterative Schema with TFX anf ML Metadata
- In this we get to know how to update an inferred schema and save the result to the metadata store used by TFX.
- The TFX components will get the information from the database before running executions.

In [1]:
!pip install tfx

In [2]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
from tfx import v1 as tfx
from tfx.types import standard_artifacts
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
from tensorflow_metadata.proto.v0 import schema_pb2
import os
import pprint
pp = pprint.PrettyPrinter()

Using TensorFlow backend


In [26]:
pipeline_path = './pipeline'
if not os.path.exists(pipeline_path):
  os.mkdir(pipeline_path)

data_root_path = './data'
if not os.path.exists(data_root_path):
  os.mkdir(data_root_path)

cenus_data_path = './data/cenus_data'
if not os.path.exists(cenus_data_path):
  os.mkdir(cenus_data_path)

In [27]:
# Define paths
_pipeline_root = './pipeline/'

# Directory of the raw data files
_data_root = './data/cenus_data/'

# path to the raw training data
_data_filepath = os.path.join(_data_root,"adult.csv")

In [28]:
!head {_data_filepath}

"age","workclass","fnlwgt","education","education.num","marital.status","occupation","relationship","race","sex","capital.gain","capital.loss","hours.per.week","native.country","income"
90,"?",77053,"HS-grad",9,"Widowed","?","Not-in-family","White","Female",0,4356,40,"United-States","<=50K"
82,"Private",132870,"HS-grad",9,"Widowed","Exec-managerial","Not-in-family","White","Female",0,4356,18,"United-States","<=50K"
66,"?",186061,"Some-college",10,"Widowed","?","Unmarried","Black","Female",0,4356,40,"United-States","<=50K"
54,"Private",140359,"7th-8th",4,"Divorced","Machine-op-inspct","Unmarried","White","Female",0,3900,40,"United-States","<=50K"
41,"Private",264663,"Some-college",10,"Separated","Prof-specialty","Own-child","White","Female",0,3900,40,"United-States","<=50K"
34,"Private",216864,"HS-grad",9,"Divorced","Other-service","Unmarried","White","Female",0,3770,45,"United-States","<=50K"
38,"Private",150601,"10th",6,"Separated","Adm-clerical","Unmarried","White","Male",0,3770,40,"

## Data Pipeline
- TFX components accepts and generates artifacts which are instances of the different artifact types TFX has configured in the metadata store.
- Everything is done by TFX we have to inspect the output of each component to know which property of the artifact you can pass on to the next component.

In [29]:
# Initialize the interactivecontext
context = InteractiveContext(pipeline_root = _pipeline_root)

In [30]:
example_gen = tfx.components.CsvExampleGen(input_base = _data_root)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 6
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## StatisticsGen

In [31]:
statistics_gen = tfx.components.StatisticsGen(
    examples = example_gen.outputs['examples']
)

context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 7
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## SchemaGen

In [32]:
schema_gen = tfx.components.SchemaGen(
    statistics = statistics_gen.outputs['statistics']
)

context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 8
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [33]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
"'""age""'",INT,required,,-
"'""capital.gain""'",INT,required,,-
"'""capital.loss""'",INT,required,,-
"'""education""'",STRING,required,,"'""education""'"
"'""education.num""'",INT,required,,-
"'""fnlwgt""'",INT,required,,-
"'""hours.per.week""'",INT,required,,-
"'""income""'",STRING,required,,"'""income""'"
"'""marital.status""'",STRING,required,,"'\'""marital.status""\''"


,Values
Domain,
"'""education""'","'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
"'""income""'","'<=50K', '>50K'"
"'\'""marital.status""\''","'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
"'\'""native.country""\''","'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
"'""occupation""'","'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
"'""race""'","'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
"'""relationship""'","'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
"'""sex""'","'Female', 'Male'"
"'""workclass""'","'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


## Curating the schema
- we are changing the age domain with some restriction and storing the changes in the metadata store.
- First the get the location of the schema by using schema.uri and then make the changes.
- Like this we can change the domain of the features can be updates as the world changes.

In [34]:
# Get the schema uri
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri

# Get the schema pbtxt file from the schemagen output
schema = tfdv.load_schema_text(os.path.join(schema_uri,"schema.pbtxt"))

In [35]:
# Restrict the range of the `age` feature
tfdv.set_domain(schema, '"age"', schema_pb2.IntDomain(name='age', min=17, max=90))

# Display the modified schema. Notice the `Domain` column of `age`.
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
"'""age""'",INT,required,,min: 17; max: 90
"'""capital.gain""'",INT,required,,-
"'""capital.loss""'",INT,required,,-
"'""education""'",STRING,required,,"'""education""'"
"'""education.num""'",INT,required,,-
"'""fnlwgt""'",INT,required,,-
"'""hours.per.week""'",INT,required,,-
"'""income""'",STRING,required,,"'""income""'"
"'""marital.status""'",STRING,required,,"'\'""marital.status""\''"


,Values
Domain,
"'""education""'","'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
"'""income""'","'<=50K', '>50K'"
"'\'""marital.status""\''","'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
"'\'""native.country""\''","'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
"'""occupation""'","'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
"'""race""'","'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
"'""relationship""'","'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
"'""sex""'","'Female', 'Male'"
"'""workclass""'","'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


## Schema Environments
- Since the model we have trained in the local environment has both input and output.
- so for predicting in the production we have to depoly the model without the output.
- For that we are creating two environments called "TRAINING" and "SERVING".

In [36]:
# Create a schema environments for training and serving
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# omit the label from the serving environment
tfdv.get_feature(schema,'"sex"').not_in_environment.append("SERVING?")

- Now freeze the curated schema and save to the local directory.
- Freezing means once the schema is frozen You can't make any changes in the schema like modifying it.

In [37]:
# path to the updated schema directory
_updated_path = f"{_pipeline_root}/updated_schema"

# Create the directory
if not os.path.exists(_updated_path):
  os.mkdir(_updated_path)

# path to the schema file
schema_file = os.path.join(_updated_path,"schema.pbtxt")

# Save the curated schema to the schema.pbtxt file
tfdv.write_schema_text(schema,schema_file)

## ImportSchemaGen
- schema has been saved
- need to create an artifact in the metadata store that will point to the curated schema file.
- This function is used to import the curated schema to ML Metadata.

In [38]:
user_schema_importer = tfx.components.ImportSchemaGen(schema_file = schema_file)
context.run(user_schema_importer,enable_cache = False)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 9
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [39]:
context.show(user_schema_importer.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
"'""age""'",INT,required,,min: 17; max: 90
"'""capital.gain""'",INT,required,,-
"'""capital.loss""'",INT,required,,-
"'""education""'",STRING,required,,"'""education""'"
"'""education.num""'",INT,required,,-
"'""fnlwgt""'",INT,required,,-
"'""hours.per.week""'",INT,required,,-
"'""income""'",STRING,required,,"'""income""'"
"'""marital.status""'",STRING,required,,"'\'""marital.status""\''"


,Values
Domain,
"'""education""'","'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
"'""income""'","'<=50K', '>50K'"
"'\'""marital.status""\''","'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
"'\'""native.country""\''","'?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia'"
"'""occupation""'","'?', 'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
"'""race""'","'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
"'""relationship""'","'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
"'""sex""'","'Female', 'Male'"
"'""workclass""'","'?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


## ExampleValidator
- use the new artifact as the input to the other components of the pipeline.

In [40]:
example_validator = tfx.components.ExampleValidator(
    statistics = statistics_gen.outputs['statistics'],
    schema = user_schema_importer.outputs['schema']
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 10
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [41]:
context.show(example_validator.outputs['anomalies'])

## Practice with ML Metadata
- tracking artifacts and how they are related to each other.
- it records the info about a specific pipeline run , will be able to track the inputs and outputs of a particular artifacts.

In [42]:
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2

# Connect config to connect to the context's metadata store
connection_config = context.metadata_connection_config

# Instantiate a MetadataStore instance with the connection config
store = mlmd.MetadataStore(connection_config)


In [43]:
# Get the artifacts
artifact_types = store.get_artifact_types()

# Print the names
[artifact.name for artifact in artifact_types]

['Examples', 'ExampleStatistics', 'Schema', 'ExampleAnomalies']

In [44]:
# Get the artifacts types
schema_list = store.get_artifacts_by_type('Schema')

[(f"schema uri : {schema.uri}  and schema id : {schema.id}") for schema in schema_list]

['schema uri : ./pipeline/SchemaGen/schema/3  and schema id : 3',
 'schema uri : ./pipeline/ImportSchemaGen/schema/4  and schema id : 4',
 'schema uri : ./pipeline/SchemaGen/schema/8  and schema id : 8',
 'schema uri : ./pipeline/ImportSchemaGen/schema/9  and schema id : 9']

- Let's get the first instance of ExampleAnomalies to get the output of ExampleValidator

In [45]:
# Get 1st instance of ExampleAnomalies
example_anomalies = store.get_artifacts_by_type('ExampleAnomalies')[0]

# Print the artifact id
print(f'Artifact id: {example_anomalies.id}')

Artifact id: 5


- Use the artifact id to get the events

In [46]:
# Get first event related to the ID
anomalies_id_event = store.get_events_by_artifact_ids([example_anomalies.id])[0]

# Print results
print(anomalies_id_event)

artifact_id: 5
execution_id: 5
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1696832893985



- The type will be an output because this is the output of the examplevalidator component.
- We can track it through the execution id.

In [47]:
# Get execution id
anomalies_execution_id = anomalies_id_event.execution_id

# Get the events by the execution ID
event_execution = store.get_events_by_execution_ids([anomalies_execution_id])

print(event_execution)

[artifact_id: 2
execution_id: 5
path {
  steps {
    key: "statistics"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1696832893962
, artifact_id: 4
execution_id: 5
path {
  steps {
    key: "schema"
  }
  steps {
    index: 0
  }
}
type: INPUT
milliseconds_since_epoch: 1696832893963
, artifact_id: 5
execution_id: 5
path {
  steps {
    key: "anomalies"
  }
  steps {
    index: 0
  }
}
type: OUTPUT
milliseconds_since_epoch: 1696832893985
]


- we see the artifacts which marked as INPUT above representing the statistics and schema inputs.
- extract their IDs programmatically.

In [50]:
# Filter input type events
input_to_eval = [
    event.artifact_id for event in event_execution if event.type == metadata_store_pb2.Event.INPUT
]

print(input_to_eval)

[2, 4]
